<a href="https://colab.research.google.com/github/MoustHolmes/AMAS_Project/blob/main/Wandb_sweep_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

In [13]:
!pip3 install box2d-py
!pip install wandb

In [14]:
!wandb login

wandb: Currently logged in as: moustholmes (use `wandb login --relogin` to force relogin)


## Define the Q Learning Agent

In [15]:
%%writefile DQN_Agent.py
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class DeepQNetwork(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, 
            n_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)

        return actions

class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions,
            max_mem_size=100000, eps_min=0.05, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_cntr = 0
        self.iter_cntr = 0
        self.replace_target = 100

        self.Q_eval = DeepQNetwork(lr, n_actions=n_actions, input_dims=input_dims,
                                    fc1_dims=256, fc2_dims=256)
        self.Q_next = DeepQNetwork(lr, n_actions=n_actions, input_dims=input_dims,
                                    fc1_dims=64, fc2_dims=64)

        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

    def store_transition(self, state, action, reward, state_, terminal):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = terminal

        self.mem_cntr += 1

    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation]).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)

        return action

    def learn(self):
        if self.mem_cntr < self.batch_size:
            return

        self.Q_eval.optimizer.zero_grad()
        
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)

        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)

        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0

        q_target = reward_batch + self.gamma*T.max(q_next,dim=1)[0]

        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()

        self.iter_cntr += 1
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min \
                       else self.eps_min

        #if self.iter_cntr % self.replace_target == 0:
        #   self.Q_next.load_state_dict(self.Q_eval.state_dict())

Overwriting DQN_Agent.py


## Train and Test

In [28]:
%%writefile main.py

import numpy as np
import operator
import wandb
from DQN_Agent import Agent
import argparse
import gym 


def main():
    wandb.init(project='AMAS_Project', config=args)
    env = gym.make('LunarLander-v2')
    agent = Agent(gamma=0.99, epsilon=1.0, batch_size=64, n_actions=4, eps_min=0.01,
                  input_dims=[8], lr=0.001)
    avg_len = 50
    burn_in_time = 50
    episodes=250
    best_avg_score, weighted_best_score = train(env, agent, episodes = episodes, avg_len = avg_len, burn_in_time = burn_in_time)

    wandb.log({'best_avg_score': best_avg_score,'weighted_best_score':weighted_best_score})

def train(env, agent, episodes=500, avg_len = 50, burn_in_time = 50):
    """The play function runs iterations and updates Q-values if desired."""

    
    scores, eps_history = [], []
    best_score = float('-inf')

    for i in range(episodes):
        score = 0
        done = False
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.store_transition(observation, action, reward, 
                                    observation_, done)
            agent.learn()
            observation = observation_
        scores.append(score)
        eps_history.append(agent.epsilon)

        avg_score = np.mean(scores[-avg_len:])
        if avg_score > best_score and i > burn_in_time:
          best_score = avg_score

        print('episode ', i, 'score %.1f' % score,
            'average score %.1f' % avg_score, 'best score:%.1f' %best_score)
  
        wandb.log({'Scores': score,'Avg_Score': avg_score, 'episodes': episodes})

    weighted_best_score =np.max( np.array(scores[burn_in_time:])/np.arange(len(scores))[burn_in_time:]) 
    print(weighted_best_score)
    return best_score, weighted_best_score

def argumentParser():
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--eps_min', default=0.01, type=float, help='Probability of chossing random action')
    parser.add_argument('--eps_dec', default=0.01, type=float, help='Probability of chossing random action')
    parser.add_argument('--lr', default=0.001, type=float, help='Learning Rate')
    parser.add_argument('--gamma', default=0.95, type=float, help='Discounting Factor')
    parser.add_argument('--Episodes', default=250, type=float, help='number of episodes')
    parser.add_argument('--burn_in_time', default=50, type=float, help='number of episodes before calculating avg score')
    parser.add_argument('--avg_len', default=50, type=float, help='number of episodes avg is calculated over')

    return parser

if __name__ == '__main__':
  global args
  args = argumentParser().parse_args()
  main()

Overwriting main.py


In [29]:
!python3 main.py

wandb: Currently logged in as: moustholmes (use `wandb login --relogin` to force relogin)
2021-03-20 10:46:49.068608: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.22
wandb: Syncing run volcanic-waterfall-13
wandb: ⭐️ View project at https://wandb.ai/moustholmes/AMAS_Project
wandb: 🚀 View run at https://wandb.ai/moustholmes/AMAS_Project/runs/2dqqk1m3
wandb: Run data is saved locally in /content/wandb/run-20210320_104647-2dqqk1m3
wandb: Run `wandb offline` to turn off syncing.

episode  0 score -53.3 average score -53.3 best score:-inf
episode  1 score -111.8 average score -82.6 best score:-inf
episode  2 score -223.7 average score -129.6 best score:-inf
episode  3 score -78.9 average score -116.9 best score:-inf
episode  4 score -84.9 average score -110.5 best score:-inf
episode  5 score -262.8 average score -135.9 best score:-inf
episode  6 score -135.0 average score -13

## Sweep for Hyperparameter tuning

In [30]:
%%writefile sweep.yaml
project: "AMAS_Project"
program: main.py
method: bayes
metric:
  name: best_score
  goal: maximize
parameters:
  lr:
    min: 0.00001
    max: 1
  gamma:
    min: 0.00001
    max: 1
  eps_dec:
    min: 0.00001
    max: 1
  eps_min:
    min: 0.00001
    max: 1

Overwriting sweep.yaml


In [31]:
!wandb sweep sweep.yaml

wandb: Creating sweep from: sweep.yaml
wandb: Created sweep with ID: 4smgq0un
wandb: View sweep at: https://wandb.ai/moustholmes/AMAS_Project/sweeps/4smgq0un
wandb: Run sweep agent with: wandb agent moustholmes/AMAS_Project/4smgq0un


In [32]:
!wandb agent moustholmes/AMAS_Project/4smgq0un

wandb: Starting wandb agent 🕵️
2021-03-20 10:58:37,523 - wandb.wandb_agent - INFO - Running runs: []
2021-03-20 10:58:37,768 - wandb.wandb_agent - INFO - Agent received command: run
2021-03-20 10:58:37,768 - wandb.wandb_agent - INFO - Agent starting run with config:
	eps_dec: 0.9474311607209955
	eps_min: 0.9028462658349244
	gamma: 0.9472946885237398
	lr: 0.7418546497278676
2021-03-20 10:58:37,769 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python main.py --eps_dec=0.9474311607209955 --eps_min=0.9028462658349244 --gamma=0.9472946885237398 --lr=0.7418546497278676
wandb: Currently logged in as: moustholmes (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
2021-03-20 10:58:39.797695: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.22
wandb: Syncing run flowing-sweep-1
wandb: ⭐️ View project at

KeyboardInterrupt: ignored